# Introduction

If you were to generate an image for every possible arrangement of pixels, then you would have every single image that has ever been created, _or ever will_.

Could you dig out images from the past?

_From the future?_

Could you find people who never existed?

_Could you find yourself?_

# A working model

In this model, each pixel contains a single integer value which denotes its color.

For example, an 2x2 image with 2 colors may look like this:

    00
    00

There are 16 possible permutations for this image, the other 15 are:

    10  01  11  00  10  01  11  00  10  01  11  00  10  01  11
    00  00  00  10  10  10  10  01  01  01  01  11  11  11  11

## Calculating permutations

If $c$ is the maximum number of colors and $p$ is the total number of pixels, then each pixel has $c$ possible values, and $c$ permutations with the other pixels in the image, which means that the total number of permutations is simply $c^{p}$.

In [1]:
# A 2x2 image with 2 colors
4 ** 2

16

## Generating pixel data

If you look at the order of the permutations in our example 2x2 image above you'll notice that the pixels increment linearly and sequentially, making it trivial to devise an algorithm to determine their values.

When generated sequentially like this, each pixel cycles through all of its color values in succession before wrapping back around to zero when its next neighbour is incremented. This continues until every pixel is equal to the maximum color value.

The first pixel increments every permutation, but the second pixel increments only when the first pixel has completed its cycle and reset to zero and so it takes two permutations to change value. The third pixel increments only when the _second_ pixel has completed its cycle and reset to zero and so it takes _four_ permutations to change value.

So if $c$ is the maximum number of colors and $i$ is the index position of a given pixel in an image, then the rate at which it increments can be determined by $c^{i}$.

In [2]:
# The first pixel increments every 1 permutation
2 ** 0

1

In [3]:
# The second pixel increments every 2 permutations
2 ** 1

2

In [4]:
# The third pixel increments every 4 permutations
2 ** 2

4

So if $p$ is the index of the permutation in the sequence, $c$ is the maximum number of colors and $i$ is the index position of a given pixel, then its value can be determined by the integer value of $\frac{p}{c^{i}}$.

We can test this out by generating the first couple of pixels of the second permutation:

In [5]:
# The first pixel of the second permutation is a 1
int(1 / 2 ** 0)

1

In [6]:
# The second pixel of the second permutation is a 0
int(1 / 2 ** 1)

0

## Generating image data

Our formula for determining the number of permutations as a function:

In [7]:
def num_permutations(num_colors, num_pixels):
    return num_colors ** num_pixels

Our formula for determining a pixel value as a function:

In [8]:
def get_pixel(index, pixel_index, num_pixels, num_colors):
    return int((index / (num_colors ** pixel_index)) % num_colors)

And a function to generate all the pixel values for a given image:

In [9]:
def get_image(index, num_pixels, num_colors):
    return list(map(lambda i: get_pixel(index, i, num_pixels, num_colors), range(0, num_pixels)))

We can test these out by grabbing some permutations from our 4x4 example image for comparison:

In [10]:
# Get the first image
get_image(0, 4, 2)

[0, 0, 0, 0]

In [11]:
# Get the second image
get_image(1, 4, 2)

[1, 0, 0, 0]

In [12]:
# Get the third image
get_image(2, 4, 2)

[0, 1, 0, 0]

## Generating an image

Now we've got all the parts, we can try generating an _actual_image:

In [13]:
from PIL import Image

def generate_image(data, image_size, num_colors):
    image = Image.new('RGB', (image_size, image_size))
    
    for y in range(0, image_size):
        for x in range(0, image_size):
            color = int((255 / num_colors) * data[y * image_size + x])
            image.putpixel((x, y), (color, color, color))
    
    return image

In [14]:
from random import random

image_size = 16
num_colors = 2

# Get a random permutation index
permutation = int((num_permutations(num_colors, image_size * image_size) / 1) * random())

# Get the data for that permutation
data = get_image(permutation, image_size * image_size, num_colors)

# Generate an image
generate_image(data, image_size, num_colors)

This is permutation 62088613102414677828980576551359319042559189409217857016239092469159532953600


## The problem

Things start to go awry as you increase the size and color depth of the image. A tiny image of just 32x32 pixels and 8 colors has an incredible number of permutations:

In [15]:
8 ** (32 * 32)

5809605995369958062859502533304574370686975176362895236661486152287203730997110225737336044533118407251326157754980517443990529594540047121662885672187032401032111639706440498844049850989051627200244765807041812394729680540024104827976584369381522292361208779044769892743225751738076979568811309579125511333093243519553784816306381580161860200247492568448150242515304449577187604136428738580990172551573934146255830366405915000869643732053218566832545291107903722831634138599586406690325959725187447169059540805012310209639011750748760017095360734234945757416272994856013308616958529958304677637019181594088528345061285863898271763457294883546638879554311615446446330199254382340016292057090751175533888161918987295591531536698701292267685465517437915790823154844634780260102891718032495396075041899485513811126977307478969074857043710716150121315922024556759241239013152919710956468406379442914941614357107914462567329693696

And if we stretch for something _barely_ usable, this happens:

In [16]:
8 ** (128 * 128)

1684018569085003115856525219531605142436233159781333642723123981705640965893292088497933711583713654956392080910520825204018621604016583770934377618636093229827983321247312546088197199743332668693742272500725383380333469433835886329196477176509366829365320004499004600332780924584477578833244223264809967604842290847674576932551118769294760354599337350582841569894276953418034214037457233120386881993756426268487272504201500256463469830846893566404359729135104794004174130102638547949413172962811401997799288183975603075063296368571801148022520861760772759049002307626223673655900610674797943971539382976399794194509017833009289892816457747952338368756515175832524982184444364607239645077211172916685317053646070275753848921674661243612344829297962359011326833261578808158439219609897555062601124827741353243480799197790383361811958008693937373728956428066044436640481995468718843700243001474316060553164633666694872657926307631362994147922142791937107754849131759440166613654952023006626916204185080

## The goal

Ideally we want to be able to generate images of _at least_ 256x256 pixels with 256 colors. I'll let you do the mathematics on the number of permutations. Bear in mind the image generation example above won't even work with an image that size because the numbers are simply too large for the computer to deal with.

The puzzle is simply to devise a way to allow _all_ permutations to be referenced and generated by an index.